In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from scipy.optimize import curve_fit
import h5py
import pyvisa
import time
from scipy.signal import savgol_filter
import scipy.signal as signal
from scipy import interpolate
from matplotlib.ticker import MaxNLocator
from scipy.optimize import least_squares
import winsound

In [2]:
factor = 2
SMALL_SIZE = factor*8
MEDIUM_SIZE = factor*10
BIGGER_SIZE = factor*12
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [3]:
rm = pyvisa.ResourceManager()
rm.list_resources()

('USB0::0x1AB1::0x04CE::DS1ZA204519116::INSTR',
 'ASRL4::INSTR',
 'ASRL5::INSTR',
 'ASRL6::INSTR',
 'ASRL8::INSTR')

In [4]:
for i in range(2):
    try:
        HV_control = rm.open_resource('ASRL6::INSTR')
        print(HV_control.query("?"))
    except pyvisa.VisaIOError:
        pass

HV_control v1.3 ready. Board B.

HV_control v1.3 ready. Board B.



In [21]:
HV_control.close()

In [39]:
DMM = rm.open_resource('GPIB0::22::INSTR')
DMM.read_termination = "\r\n"
DMM.timeout = 5000

In [4]:
scope = rm.open_resource('USB0::0x1AB1::0x04CE::DS1ZA204519116::INSTR')
scope.query("*IDN?")

'RIGOL TECHNOLOGIES,DS1104Z,DS1ZA204519116,00.04.04.SP3\n'

### Data collection functions

In [5]:
def get_data(t_total, do_tqdm, NPLC):
    # start the measurement
    DMM.clear()
    DMM.write(f"NDIG 8")
    DMM.write(f"NPLC {NPLC}")
    DMM.write("TARM AUTO")
    
    # collect data
    data = []
    t0 = time.time()
    if do_tqdm:
        pbar = tqdm(total=t_total)

    while time.time()-t0 < t_total:
        # get data from HV_control board
        data_str = HV_control.query('a')
        vals_arr = [(int(x)-32768) / 65535 * (2 * 3 * 4.096) for x in data_str.split(',')]
        
        data.append([time.time(), *vals_arr, float(DMM.read())])
        print(f"{10*data[-1][4]:.3f} kV, {data[-1][-1]:.9f} mV", end='\r')
        if do_tqdm:
            pbar.update(int(time.time()-t0)+1 - pbar.n)

    if do_tqdm:
        pbar.close()
    data = np.array(data)
    
    return data
    
def DMM_to_HDF(run_name, t_total, fname="test.hdf", overwrite=False, do_tqdm=False, NPLC=10, comment=""):
    # check that a dataset with this run_name doesn't already exist
    try:
        with h5py.File(fname, 'a') as f:
            if run_name in f.keys():
                if overwrite:
                    del f[run_name]
                else:
                    print("ERROR: Run name already exists.")
                    return
    except OSError:
        pass
    
    # get data
    data = get_data(t_total, do_tqdm, NPLC)
    
    # save to HDF
    with h5py.File(fname, 'a') as f:
        dset = f.create_dataset(run_name, data=data)
        dset.attrs['timestamp'] = time.time()
        dset.attrs['overwritten'] = str(overwrite)
        dset.attrs['t_total'] = t_total
        dset.attrs['DMM_temp'] = float(DMM.query("TEMP?"))
        dset.attrs['NPLC'] = NPLC
        dset.attrs['comment'] = comment

In [6]:
def ramp(V_arr, dt=.1, printing=True, beep=False, do_tqdm=True):
    # whether to display a progress bar
    if do_tqdm:
        tqdm_fn = tqdm
    else:
        tqdm_fn = lambda x:x
        
    for V_set in tqdm_fn(V_arr):
        if beep:
            winsound.Beep(1000, 500)
        if printing:
            print(f"{int(V_set)} = {V_set/59055*30:.2f} kV     ", end="\r")
        HV_control.write(f'd{int(V_set)}')
        time.sleep(dt)

In [7]:
def measure_AC(run_name, fname, freq_arr, polarity="positive", amp=2000, offs=10000, dt0=60, overwrite=False):
    # check that a dataset with this run_name doesn't already exist
    try:
        with h5py.File(fname, 'a') as f:
            if run_name in f.keys():
                if overwrite:
                    del f[run_name]
                else:
                    print("ERROR: Run name already exists.")
                    return
    except OSError:
        pass

    # set TOF3000
    HV_control.write('h1');
    HV_control.write('r1');
    HV_control.write('d0');
    if polarity == "positive":
        HV_control.write('p1');
    elif polarity == "negative":
        HV_control.write('p1');
    HV_control.write('h0');
    ramp(np.linspace(0, offs, 100), do_tqdm=False)
    
    # get data
    data = []
    dt_arr = [int(x) for x in dt0/np.array(freq_arr)]
    try:
        for freq,dt in zip(tqdm(freq_arr), dt_arr):
            scope.write(f":TIM:SCAL {dt/24}")
            scope.write(f":TIM:OFFS {0}")
            HV_control.write(f"A0; S1,{freq},{amp},{offs}");
            for i in range(3*dt):
                time.sleep(1)
            try:
                Vpp = float(scope.query(":MEAS:ITEM? Vpp,CHAN2"))
                f_meas = float(scope.query(":MEAS:ITEM? frequency,CHAN2"))
            except ValueError:
                Vpp, f_meas = np.nan, np.nan
            data.append([freq, f_meas, Vpp])
            print(data[-1], end='          \r')
    except KeyboardInterrupt:
        pass
        
    # turn off TOF3000
    HV_control.write(f"A1; S0");
    ramp(np.linspace(offs, 0, 100), do_tqdm=False)
    HV_control.write('d0');
    HV_control.write('h1');
    
    # save to HDF
    with h5py.File(fname, 'a') as f:
        dset = f.create_dataset(run_name, data=data)
        dset.attrs['timestamp'] = time.time()
        dset.attrs['overwritten'] = str(overwrite)
        dset.attrs['polarity'] = polarity
        dset.attrs['amp'] = amp
        dset.attrs['offs'] = offs
        dset.attrs['dt0'] = dt0

### Testing

In [78]:
print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))
print(DMM.query("TEMP?"))
try:
    DMM.write("ACAL DCV")
except pyvisa.errors.VisaIOError:
    pass

2023-04-14 11:00:10
35.1


### Positive polarity

In [20]:
HV_control.write('p1');
HV_control.write('r1');
HV_control.write('h1');
HV_control.write('d0');

In [19]:
HV_control.write('h0');
HV_control.write(f'd{int(59055 / 30 * 0)}');

In [14]:
ramp(59055 / 30 * np.linspace(0, 20, 200))

  0%|          | 0/200 [00:00<?, ?it/s]

In [17]:
ramp(59055 / 30 * np.linspace(20, 30, 200))

  0%|          | 0/100 [00:00<?, ?it/s]

In [18]:
ramp(59055 / 30 * np.linspace(30, 0, 300))

  0%|          | 0/300 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [40]:
try:
    for i in tqdm(range(9999)):
        DMM_to_HDF(
            run_name  = f"A-{i}",
            t_total   = 60,
            NPLC      = 1,
            fname     = "closed_box.hdf",
            overwrite = True,
            do_tqdm   = False,
            comment   = "",
        )
except KeyboardInterrupt:
    pass

  0%|          | 0/9999 [00:00<?, ?it/s]

In [130]:
HV_control.write('r1');

In [129]:
ramp(59055 / 30 * np.linspace(20, 0, 100))

  0%|          | 0/100 [00:00<?, ?it/s]

### Negative polarity

In [178]:
HV_control.write('p0');
HV_control.write('r1');
HV_control.write('h1');
HV_control.write('d0');

In [219]:
HV_control.write('h0');
HV_control.write(f'd{int(59055 / 30 * 28.1201)}');

In [180]:
ramp(59055 / 30 * np.linspace(0, 20, 1000))

  0%|          | 0/1000 [00:00<?, ?it/s]

In [204]:
ramp(59055 / 30 * np.linspace(25, 31, 500))

  0%|          | 0/500 [00:00<?, ?it/s]

In [205]:
ramp(59055 / 30 * np.linspace(31, 25, 100))

  0%|          | 0/100 [00:00<?, ?it/s]

In [197]:
HV_control.write('r0');

In [198]:
HV_control.write('r1');

In [223]:
try:
    for i in tqdm(range(9999)):
        DMM_to_HDF(
            run_name  = f"Z-{i}",
            t_total   = 60,
            NPLC      = 1,
            fname     = "HV_leakage_test.hdf",
            overwrite = True,
            do_tqdm   = False,
            comment   = "",
        )
except KeyboardInterrupt:
    pass

  0%|          | 0/9999 [00:00<?, ?it/s]

In [176]:
HV_control.write('r1');

In [224]:
ramp(59055 / 30 * np.linspace(28.1201, 0, 500))

  0%|          | 0/500 [00:00<?, ?it/s]

### AC measurements

In [364]:
measure_AC(
    run_name  = "B-1",
    fname     = "HV_AC_test.hdf",
    freq_arr  = [int(x) for x in np.geomspace(500,900,25)][1:],
    polarity  = "positive",
    amp       = 1000,
    offs      = 49000,
    dt0       = 3600,
    overwrite = True,
)

  0%|          | 0/24 [00:00<?, ?it/s]